# 02 Calibration and Decision Costs

Calibration curves, Brier score, and cost-based thresholds.


## Table of Contents
- Calibration
- Brier score
- Decision costs


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Matching Guide
- `docs/guides/03_classification/02_calibration_and_costs.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/03_classification/02_calibration_and_costs.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Calibrate probabilities and choose thresholds based on decision costs.


## Your Turn: Calibration


In [ ]:
# TODO: Compute calibration curve and Brier score
# Compare calibrated vs uncalibrated probabilities
...


## Your Turn: Cost-based threshold


In [ ]:
# TODO: Define a simple cost matrix (FP vs FN)
# Pick a threshold that minimizes expected cost
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Calibration</summary>

```python
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Assume y_te and p from prior notebook
prob_true, prob_pred = calibration_curve(y_te, p, n_bins=6)
plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1], linestyle='--', color='gray')
plt.xlabel('Predicted')
plt.ylabel('Observed')
plt.title('Calibration curve')
plt.show()
```

</details>

<details><summary>Solution: Brier score</summary>

```python
from sklearn.metrics import brier_score_loss
brier_score_loss(y_te, p)
```

</details>

<details><summary>Solution: Decision costs</summary>

```python
import numpy as np

# Simple cost model: cost_fp and cost_fn
cost_fp = 1.0
cost_fn = 5.0

def expected_cost(thr):
    pred = (p >= thr).astype(int)
    fp = ((pred == 1) & (y_te.to_numpy() == 0)).sum()
    fn = ((pred == 0) & (y_te.to_numpy() == 1)).sum()
    return cost_fp*fp + cost_fn*fn

candidates = np.linspace(0.05, 0.95, 19)
best = min([(thr, expected_cost(thr)) for thr in candidates], key=lambda t: t[1])
best
```

</details>

